# Clean webtext from charter school data

- Author: Jaren Haber, Madeleine Peng, James Jung
- Institution: UC Berkeley
- Date created: July 26, 2019
- Date last edited: 

Description: 

## Initialize

In [5]:
# Import packages
import pandas as pd # For working with DataFrames
import gc # To accelerate loading pickle files
import nltk
import re, datetime
import string # for one method of eliminating punctuation
from nltk.corpus import stopwords # for eliminating stop words
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer; ps = PorterStemmer() # approximate but effective (and common) method of stemming words
import os # for working with file trees
import numpy as np
import spacy 
from __future__ import unicode_literals

# Show visualizations within notebook:
%matplotlib inline 

In [116]:
# Load functions from data_tools directory:
import sys; sys.path.insert(0, "../tools")

# For displaying basic DF info, storing DFs for memory efficiency, and loading a filtered DF:
from df_tools import check_df, convert_df, load_filtered_df, replace_df_nulls

# For quickly loading & saving pickle files in Python:
from quickpickle import quickpickle_dump, quickpickle_load 

# For saving and loading text lists to/from file:
from textlist_file import write_list, load_list 

In [124]:
# Set file paths
charters_path = "../../nowdata/charters_2015.pkl"
original_path = "../../nowdata/backups/charters_full_2015_250_v2a_unlappedtext_counts3_geoleaid.pkl"
filtered_path = "../../nowdata/parsing/filtered_10.pkl"
raw_folder = "../../nowdata/webtext_raw/" # for raw webtext when extracted
raw_filtered_data = raw_folder + "webtext_unlapped_filtered_10.tsv"
cleaned_folder = "../../nowdata/webtext_cleaned/" # for cleaned webtext: save as CSV, include ONLY the columns "NCESSCH" (unique school identifier) and "text_full" (renamed from "WEBTEXT")

In [118]:
#Import spacy and clean_text to test clean_sentence
import spacy, clean_text
from clean_text import clean_sentence

In [98]:
print("ABOUT\nSCHOOLS\nPROGRAMS\nENROLL\nPARENT LINKS\nCOMMUNITY\nCONTACT\nMORE>>\nBENNETT ACADEMY ACCELERATED HIGH SCHOOL\xa0\n\u200b\nBennett Accelerated High School\xa0\nBennett Accelerated High School will begin serving students in 9th and 10th grade in August 2018. Bennett Accelerated High School will offer the followng:\xa0\n\u200b\nA good school with good discipline\nA well-organized learning environment with a friendly vibe\nA place where teachers, staff, and students treat each other kindly and respectfully\nAn environment with friends (and we add, good friends)\nTeachers who care about their students and give them good help\nCourses that teach them what they need\nCore classes and several electives\nSports\nVision and Purpose\nGrade 9 Course Descriptions\nGrade 10 Course Descriptions\nFAQ\nEnroll Now\n© 2012 by Twenty First\xa0Century\xa0\n\xa0 \xa0 \xa0Charter Schools\n\u200b\nBennett Academy Middle School\n\u200b2930 W. Bethany Home Road. Phoenix, AZ\xa085017\nBennett Academy Primary School - Venture Site\n1535 W.\xa0Dunlap Ave. Phoenix, AZ 85021\nWebmaster Login\n\ufeffPursuant to A.R.S. §38-431.02, Twenty First\xa0Century\xa0\nSchools, Inc. hereby states that all notices of the meetings of Bennett Academy and Bennett Venture Site will be posted at each administrative office and on our website (www.bennettacademy.com). The location is open to the public Monday through Friday from 8:00 AM to 4:00 PM. Such notices will indicate the date, time and place of the meeting and will include an agenda or information concerning the manner in which the public may obtain an agenda for the meeting.\n\u200b")

ABOUT
SCHOOLS
PROGRAMS
ENROLL
PARENT LINKS
COMMUNITY
CONTACT
MORE>>
BENNETT ACADEMY ACCELERATED HIGH SCHOOL 
​
Bennett Accelerated High School 
Bennett Accelerated High School will begin serving students in 9th and 10th grade in August 2018. Bennett Accelerated High School will offer the followng: 
​
A good school with good discipline
A well-organized learning environment with a friendly vibe
A place where teachers, staff, and students treat each other kindly and respectfully
An environment with friends (and we add, good friends)
Teachers who care about their students and give them good help
Courses that teach them what they need
Core classes and several electives
Sports
Vision and Purpose
Grade 9 Course Descriptions
Grade 10 Course Descriptions
FAQ
Enroll Now
© 2012 by Twenty First Century 
     Charter Schools
​
Bennett Academy Middle School
​2930 W. Bethany Home Road. Phoenix, AZ 85017
Bennett Academy Primary School - Venture Site
1535 W. Dunlap Ave. Phoenix, AZ 85021
Webmaster Logi

## Import sentence cleaning functions

In [ ]:
# Prep dictionaries of English words
from nltk.corpus import words # Dictionary of 236K English words from NLTK
english_nltk = set(words.words()) # Make callable
english_long = set() # Dictionary of 467K English words from https://github.com/dwyl/english-words
fname =  "/home/jovyan/work/data_management/tools/english-words.txt" # Set file path to long english dictionary
with open(fname, "r") as f:
    for word in f:
        english_long.add(word.strip())

#to look for proper nouns when cleaning text
from nltk.tag import pos_tag

In [ ]:
def stopwords_make(vocab_path_old = "", extend_stopwords = False):
    """Create stopwords list. 
    If extend_stopwords is True, create larger stopword list by joining sklearn list to NLTK list."""
                                                     
    stop_word_list = list(set(stopwords.words("english"))) # list of english stopwords

    # Add dates to stopwords
    for i in range(1,13):
        stop_word_list.append(datetime.date(2008, i, 1).strftime('%B'))
    for i in range(1,13):
        stop_word_list.append((datetime.date(2008, i, 1).strftime('%B')).lower())
    for i in range(1, 2100):
        stop_word_list.append(str(i))

    # Add other common stopwords
    stop_word_list.append('00') 
    stop_word_list.extend(['mr', 'mrs', 'sa', 'fax', 'email', 'phone', 'am', 'pm', 'org', 'com', 
                           'Menu', 'Contact Us', 'Facebook', 'Calendar', 'Lunch', 'Breakfast', 
                           'facebook', 'FAQs', 'FAQ', 'faq', 'faqs', '1', '2', '3', '4', '5', '6',
                          '7','8','9','0']) # web stopwords + numbers
    stop_word_list.extend(['el', 'en', 'la', 'los', 'para', 'las', 'san']) # Spanish stopwords
    stop_word_list.extend(['angeles', 'diego', 'harlem', 'bronx', 'austin', 'antonio']) # cities with many charter schools

    # Add state names & abbreviations (both uppercase and lowercase) to stopwords
    states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 
              'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 
              'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 
              'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 
              'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WI', 'WV', 'WY', 
              'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
              'Colorado', 'Connecticut', 'District of Columbia', 'Delaware', 'Florida', 
              'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 
              'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
              'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
              'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
              'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 
              'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 
              'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 
              'Vermont', 'Virginia', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming' 
              'carolina', 'columbia', 'dakota', 'hampshire', 'mexico', 'rhode', 'york']
    for state in states:
        stop_word_list.append(state)
    for state in [state.lower() for state in states]:
        stop_word_list.append(state)
        
    # Add even more stop words:
    if extend_stopwords == True:
        stop_word_list = text.ENGLISH_STOP_WORDS.union(stop_word_list)
        
    # If path to old vocab not specified, skip last step and return stop word list thus far
    if vocab_path_old == "":
        return stop_word_list

    # Add to stopwords useless and hard-to-formalize words/chars from first chunk of previous model vocab (e.g., a3d0, \fs19)
    # First create whitelist of useful terms probably in that list, explicitly exclude from junk words list both these and words with underscores (common phrases)
    whitelist = ["Pre-K", "pre-k", "pre-K", "preK", "prek", 
                 "1st", "2nd", "3rd", "4th", "5th", "6th", "7th", "8th", "9th", "10th", "11th", "12th", 
                 "1st-grade", "2nd-grade", "3rd-grade", "4th-grade", "5th-grade", "6th-grade", 
                 "7th-grade", "8th-grade", "9th-grade", "10th-grade", "11th-grade", "12th-grade", 
                 "1st-grader", "2nd-grader", "3rd-grader", "4th-grader", "5th-grader", "6th-grader", 
                 "7th-grader", "8th-grader", "9th-grader", "10th-grader", "11th-grader", "12th-grader", 
                 "1stgrade", "2ndgrade", "3rdgrade", "4thgrade", "5thgrade", "6thgrade", 
                 "7thgrade", "8thgrade", "9thgrade", "10thgrade", "11thgrade", "12thgrade", 
                 "1stgrader", "2ndgrader", "3rdgrader", "4thgrader", "5thgrader", "6thgrader", 
                 "7thgrader", "8thgrader", "9thgrader", "10thgrader", "11thgrader", "12thgrader"]
    with open(vocab_path_old) as f: # Load vocab from previous model
        junk_words = f.read().splitlines() 
    
    if school_whitelist == True:
        junk_words = [word for word in junk_words[:8511] if ((not "_" in word) 
                                                         and (not any(term in word for term in whitelist)))]
        stop_word_list.extend(junk_words)                                      
        return stop_word_list
        
    junk_words = [word for word in junk_words[:8511] if ((not "_" in word))]
    stop_word_list.extend(junk_words)
                  
    return stop_word_list
                                                     
    

In [ ]:
def punctstr_make():
    """Creates punctuations list"""
                    
    punctuations = list(string.punctuation) # assign list of common punctuation symbols
    #addpuncts = ['*','•','©','–','`','’','“','”','»','.','×','|','_','§','…','⎫'] # a few more punctuations also common in web text
    #punctuations += addpuncts # Expand punctuations list
    #punctuations = list(set(punctuations)) # Remove duplicates
    punctuations.remove('-') # Don't remove hyphens - dashes at beginning and end of words are handled separately)
    punctuations.remove("'") # Don't remove possessive apostrophes - those at beginning and end of words are handled separately
    punctstr = "".join([char for char in punctuations]) # Turn into string for regex later

    return punctstr
                                                     
                                                     

In [ ]:
def unicode_make():
    """Create list of unicode chars"""
                    
    unicode_list  = []
    for i in range(1000,3000):
        unicode_list.append(chr(i))
    unicode_list.append("_cid:10") # Common in webtext junk
                                                     
    return unicode_list

In [ ]:
def get_common_words(tokenized_corpus, max_percentage):
    """Discover most common words in corpus up to max_percentage.
    
    Args:
        Corpus tokenized by words,
        Highest allowable frequency of documents in which a token may appear (e.g., 1-5%)
        
    Returns:
        List of most frequent words in corpus"""
    
    # Code goes here
    # Probably using nltk.CountVectorizer

    
# Create useful lists using above functions:
stop_words_list = stopwords_make()
punctstr = punctstr_make()
unicode_list = unicode_make()

# Instantiate spacy object for part of speech tagging (to remove proper nouns)
nlp = spacy.load('en', disable=['ner']) # For speed, keep only POS tagging functionality and parsing (which improves accuracy). Does including NER also improve accuracy? 

In [ ]:
def gather_propernouns(sentence):
    """ Creates a list of the propernouns in the sentence.
    Args:
        docs: Spacy object of sentence  
    Returns:
        List of proper nouns in the sentence."""
                  
    tagged_sentence = pos_tag(sentence.split())
    # [('James', 'NNP'), ('likes', 'VBZ'), ('apples', 'NNPS')]
    
    propernouns = [word for word,pos in tagged_sentence if pos == 'NNP' or pos == 'NNPS']
    # ['James', 'apples']
 
    return propernouns
#     new_text = []
#     for word in text:
#         if word.pos_ == "PROPN":
#             new_text.append(str(word))
#     print(new_text) # while debugging
#     return new_text

In [109]:
def clean_sentence(sentence, remove_stopwords = True, remove_numbers = True, keep_english = False, slow_webclean = True, exclude_words = [], stemming = False, remove_acronyms = False, remove_propernouns = False, unhyphenate = False, return_string = False):
    """Removes numbers, emails, URLs, unicode characters, hex characters, and punctuation from a sentence 
    separated by whitespaces. Returns a tokenized, cleaned list of words from the sentence.
    
    Args: 
        sentence, i.e. string that possibly includes spaces and punctuation
        remove_stopwords: whether to remove stopwords, default True
        remove_numbers: whether to remove any chars that are digits, default True
        keep_english: whether to remove words not in english dictionary, default False; if 'restrictive', keep word only if in NLTK's dictionary of 237K english words; if 'permissive', keep word only if in longer list of 436K english words
        slow_webclean: whether to include advanced sentence cleaning: removing emails, URLs, and unicode and hex chars, default False
        exclude_words: list of words to exclude, may be most common words or named entities, default empty list
        stemming: whether to apply PorterStemmer to each word, default False
        remove_propernouns: boolean, removes nouns such as names, etc., default True 
        unhyphenate: join any lingering hyphens (i.e., words ending with '- ') as for JSTOR text data, default False
        return_string: return string instead of list of tokens (useful for infersent), default False
    Returns: 
        Cleaned & tokenized sentence, i.e. a list of cleaned, lower-case, one-word strings"""
    
    global stop_words_list, punctstr, unicode_list, english_nltk, nlp, english_long
    
    # Replace unicode spaces, tabs, and underscores with spaces, and remove whitespaces from start/end of sentence:
    sentence = sentence.replace(u"\xa0", u" ").replace(u"\\t", u" ").replace(u"_", u" ").strip(" ")
    
    if unhyphenate:              
        ls = re.findall(r"\w+-\s\w+", sentence)
        if len(ls) > 0:
            ls_new = [re.sub(r"- ", "", word) for word in ls]
            for i in range(len(ls)):
                sentence= sentence.replace(ls[i], ls_new[i])

    if remove_acronyms:
        ls = re.findall(r"\b[A-Z][A-Z]+\b\s+", sentence)
        if len(ls) > 0:
            ls_new = np.repeat("", len(ls))
            for i in range(len(ls)):
                sentence = sentence.replace(ls[i], ls_new[i])
    
    if slow_webclean:
        # Remove hex characters (e.g., \xa0\, \x80):
        sentence = re.sub(r'[^\x00-\x7f]', r'', sentence) #replace anything that starts with a hex character 

        # Replace \\x, \\u, \\b, or anything that ends with \u2605
        sentence = re.sub(r"\\x.*|\\u.*|\\b.*|\u2605$", "", sentence)

        # Remove all elements that appear in unicode_list (looks like r'u1000|u10001|'):
        sentence = re.sub(r'|'.join(map(re.escape, unicode_list)), '', sentence)
    
    # If True, include the proper nouns in stop_words_list
    if remove_propernouns:              
#         doc = nlp(sentence) # Create a document object in spacy
#         proper_nouns = gather_propernouns(doc) # Creates a wordbank of proper nouns we should exclude
           
        #trying to gather proper nouns by passing in pure sentence in gather_propernouns
        proper_nouns = gather_propernouns(sentence)
        print(proper_nouns)
        # Remove each proper noun from sentence:
        sentence = " ".join([word for word in sentence.split() if word not in proper_nouns])
        #for term in proper_nouns: # Loop over wordbank
        #    sentence = re.sub(term, "", sentence) # Less effective because removes characters from within terms

    
    sent_list = [] # Initialize empty list to hold tokenized sentence (words added one at a time)
    
    for word in sentence.split(): # Split by spaces and iterate over words
        
        word = word.strip() # Remove leading and trailing spaces
        
        if remove_numbers:
            word = re.sub(r"[0-9]+", "", word) # filter out digits
        
        # Filter out emails and URLs:
        if slow_webclean and ("@" in word or word.startswith(('http', 'https', 'www', '//', '\\', 'x_', 'x/', 'srcimage')) or word.endswith(('.com', '.net', '.gov', '.org', '.jpg', '.pdf', 'png', 'jpeg', 'php'))):
            continue
            
        # Remove punctuation (only after URLs removed):
        word = re.sub(r"["+punctstr+"]+", r'', word).strip("'").strip("-") # Remove punctuations, and remove dashes and apostrophes only from start/end of words
        
        if remove_stopwords and word in stop_words_list: # Filter out stop words
            continue
                            
        # TO DO: Pass in most_common_words to function; write function to find the top 1-5% most frequent words, which we will exclude
        # Remove most common words:
        if word in exclude_words:
            continue
            
        if keep_english == 'restrictive':
            if word not in english_nltk: #Filter out non-English words using shorter list
                continue
            
        if keep_english == 'permissive': 
            if word not in english_long: #Filter out non-English words using longer list
                continue
        
        # Stem word (if applicable):
        if stemming:
            word = ps.stem(word)
        
        sent_list.append(word.lower()) # Add lower-cased word to list (after passing checks)
    
    if return_string:
        return ' '.join(sent_list) # Return clean, tokenized sentence (string)
    
    return sent_list # Return clean, tokenized sentence (list of tokens)

['about', 'schools', 'programs', 'enroll', 'links', 'community', 'contact', 'more', 'bennett', 'academy', 'high', 'begin', 'serving', 'students', 'th', 'th', 'grade', '', 'offer', 'followng', 'a', 'good', 'school', 'good', 'discipline', 'a', 'well-organized', 'learning', 'environment', 'friendly', 'vibe', 'a', 'place', 'teachers', 'staff', 'students', 'treat', 'kindly', 'respectfully', 'an', 'environment', 'friends', 'add', 'good', 'friends', 'teachers', 'care', 'students', 'give', 'good', 'help', 'teach', 'need', 'core', 'classes', 'several', 'electives', 'sports', '', '', 'enroll', 'now', '', '', 'road', 'phoenix', '', '', '', 'ave', 'phoenix', '', '', 'schools', 'hereby', 'states', 'notices', 'meetings', 'posted', 'administrative', 'office', 'website', 'wwwbennettacademycom', 'the', 'location', 'open', 'public', '', 'am', '', 'pm', 'such', 'notices', 'indicate', 'date', 'time', 'place', 'meeting', 'include', 'agenda', 'information', 'concerning', 'manner', 'public', 'obtain', 'agend

In [121]:
def clean_webtext(ls):
    
    '''This function cleans and tokenizes sentences, removing punctuation and numbers and making words into lower-case stems.
    Inputs: list of strings;
    This function loops over all elements in the input list given, cleans the texts and returns one string'''
        
    global mpdo # Check if we're doing multiprocessing. If so, then mpdo=True
    global sents_combined # Grants access to variable holding a list of lists of words, where each list of words represents a sentence in its original order (only relevant for this function if we're not using multiprocessing)
    global pcount # Grants access to preprocessing counter
    
    known_pages = set() # Initialize list of known pages for a school
    sents_combined = [] # Initialize list of all school's sentences
    school_sentslist = []
    #print('Parsing school #' + str(pcount)) # Print number of school being parsed

    for s in ls: # Iterate over tuples in tuplist (list of tuples)
        for chunk in s.split('\n'): 
            for sent in nltk.sent_tokenize(chunk): # Tokenize chunk by sentences (in case >1 sentence in chunk)
                #sent = clean_sentence(sent, fast=True) # Clean and tokenize sentence
                sent = clean_sentence(sent)
                if ((sent == []) or (len(sent) == 0)): # If sentence is empty, continue to next sentence without appending
                    continue
                
                sents_combined.extend(sent) # add sent to school object
                
    school_sentslist.append(sents_combined) # add list of sentence to full list 
    
    return sents_combined

## Clean webtext

In [125]:
# Load data
df = pd.read_csv(raw_filtered_data, sep="\t", encoding="utf-8")[["text_full", "NCESSCH"]]
check_df(df, "NCESSCH")

FileNotFoundError: [Errno 2] File b'../../nowdata/webtext_raw/webtext_unlapped_filtered_10.tsv' does not exist: b'../../nowdata/webtext_raw/webtext_unlapped_filtered_10.tsv'

In [123]:
df.head()

NameError: name 'df' is not defined

In [ ]:
ls_str = [] #initialize list for concatenated string for webtext per school

for i in range(len(df)):
    ls_str.append(' '.join(clean_webtext(df['text_full'][i])))
    if i%500 == 0:
        print("Cleaned ", i, " rows")
    
df['text_full'] = ls_str

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Y']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Y']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['U']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['J']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['J']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
['D']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['D']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
['M']
[]
['D']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['J']
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

...

[]
['M']
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
['S']
[]
['D']
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
['S']
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
['S']
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
['F']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['J']
[]
[]
['M']
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
['J']
[]
[]
['S']
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
['K']
['Y']
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
['K']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
['Y']
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['←']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
['K']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['D']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

[]
[]
['S']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['K']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['W']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['K']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
['U']
[]
[]
[]
[]
[]
[]
[]
['M']
[]
[]
[]
[]
[]
[]
[]
[]
[]
['F']
[]
[]
[]
[]
[]
['U']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['S']
[]
['D']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[